# Dessert and Dreams Puzzle Solver



## Introduction

The Potter family includes:
- **Mummy Potter**
- **Daddy Potter**
- **Peter (a schoolboy)**
- **Betty (younger sister)**
- **Aunt Polly**

Each family member has specific hobbies, favorite desserts, and dreams. This notebook answers the following:
1. **Who likes Napoleon Cake?**
2. **Who dreams of visiting Paris?**

## Puzzle Constraints

### Rules and Logical Statements

1. **Mummy eats only marshmallows.**
2. **Betty likes only marmalade.**
3. **Peter doesn't like anything with cream.**  
4. **Whoever likes ice cream dreams of visiting Paris.**  
5. **Peter and Betty have specific dreams:**
6. **Mummy and Daddy share the same dream.**  
7. **Family members like Mummy’s desserts: Napoleon Cake, Marmalade, and Waffles.**

In [1]:
# imports utils & logic From aima
from aima.utils import *
from aima.logic import *

def conjoin(clauses):
    
    return expr('&'.join(map(str, clauses)))

## Constraint Encoding

The following constraints are encoded in the knowledge base:
1. Mummy eats **only marshmallows**.
2. Betty likes **only marmalade**.
3. Peter dislikes cream-based desserts.
4. Whoever likes ice cream dreams of visiting Paris.
5. Specific dreams for Peter and Betty.
6. Mummy and Daddy share the same dream (Sea).
7. Napoleon Cake is liked by Daddy and Aunt Polly.


In [8]:

people = ['Mummy', 'Daddy', 'Peter', 'Betty', 'AuntPolly']
desserts = ['NapoleonCake', 'Marmalade', 'Waffles', 'Marshmallows', 'IceCream']
dreams = ['Paris', 'Sea', 'Ballet', 'CoinCollection']
holidayFavoriteDesserts = ['NapoleonCake', 'Marmalade', 'Waffles']


dessertsWithCream = ['NapoleonCake', 'IceCream']

likes = {}
for person in people:
    for dessert in desserts:
        likes[(person, dessert)] = expr(f'Likes_{person}_{dessert}')

dreams_dict = {}
for person in people:
    for dream in dreams:
        dreams_dict[(person, dream)] = expr(f'Dreams_{person}_{dream}')

kb = PropKB()

kb.tell(likes['Mummy', 'Marshmallows'])
for dessert in desserts:
    if dessert != 'Marshmallows':
        kb.tell(~likes['Mummy', dessert])

kb.tell(likes['Betty', 'Marmalade'])
for dessert in desserts:
    if dessert != 'Marmalade':
        kb.tell(~likes['Betty', dessert])
        

for dessert in dessertsWithCream:
    kb.tell(~likes['Peter', dessert])

for person in people:
    kb.tell(~likes[(person, 'IceCream')] | dreams_dict[(person, 'Paris')])



kb.tell(likes['AuntPolly', 'IceCream'])

kb.tell(dreams_dict['Peter', 'CoinCollection'])
kb.tell(dreams_dict['Betty', 'Ballet'])


kb.tell(dreams_dict['Mummy', 'Sea'])
kb.tell(dreams_dict['Daddy', 'Sea'])




for person in people:
    if person != 'AuntPolly':  
        for dessert in holidayFavoriteDesserts:  
            kb.tell(likes[(person, dessert)] | ~likes[(person, dessert)])


for person in people:
    kb.tell(dreams_dict[(person, 'Paris')] |
            dreams_dict[(person, 'Sea')] |
            dreams_dict[(person, 'Ballet')] |
            dreams_dict[(person, 'CoinCollection')])
    
    for dream1 in dreams:
        for dream2 in dreams:
            if dream1 != dream2:
                kb.tell(~dreams_dict[(person, dream1)] | ~dreams_dict[(person, dream2)])      



## Inference and Results

We use the **DPLL algorithm** to find a model that satisfies the knowledge base and query it for answers.


In [9]:
symbols = set()
for val in likes.values():
    symbols.add(val)
for val in dreams_dict.values():
    symbols.add(val)

model = dpll_satisfiable(conjoin(kb.clauses))

def who_likes_napoleon(model):
    napoleon_lover=[]
    for person in people:
        key = likes[(person, 'NapoleonCake')]
        if model.get(key, False):
            napoleon_lover.append(person)

    return napoleon_lover        

def who_dreams_paris(model):
    paris_dreamer=[]
    for person in people:
        key = dreams_dict[(person, 'Paris')]
        if model.get(key, False):
            paris_dreamer.append(person)
    return paris_dreamer        


napoleon_lover = who_likes_napoleon(model)
paris_dreamer = who_dreams_paris(model)

print(napoleon_lover, "likes Napoleon Cake")
print(paris_dreamer, "dreams of going to Paris")

['Daddy'] likes Napoleon Cake
['AuntPolly'] dreams of going to Paris


# First Order Logic

In [4]:
from aima import logic
from aima.logic import FolKB, fol_fc_ask, PropKB,dpll_satisfiable
from aima.utils import expr,Expr

In [21]:
rules = [
 'Likes(Mummy, Marshmallows)',
    'Dislikes(Mummy, NapoleonCake)',
    'Dislikes(Mummy, Marmalade)',
    'Dislikes(Mummy, Waffles)',
    'Dislikes(Mummy, IceCream)',

    'Likes(Betty, Marmalade)',
    'Dislikes(Betty, NapoleonCake)',
    'Dislikes(Betty, Marshmallows)',
    'Dislikes(Betty, Waffles)',
    'Dislikes(Betty, IceCream)',

    'ContainsCream(NapoleonCake)',
    'ContainsCream(IceCream)',
    'ContainsCream(x) ==> Dislikes(Peter, x)',

    'Likes(AuntPolly, IceCream)',
    'Likes(x, IceCream) ==> Dreams(x, Paris)',

    'Dreams(Peter, CoinCollection)',
    'Dreams(Betty, Ballet)',
    'Dreams(Mummy, Sea)',
    'Dreams(Daddy, Sea)',

    'HolidayDessert(NapoleonCake)',
    'HolidayDessert(Marmalade)',
    'HolidayDessert(Waffles)',
 
    'HolidayDessert(x) ==> Likes(Daddy, x)',

    'HolidayDessert(x) & Dislikes(y, x) ==> NotLikes(y, x)',
    'HolidayDessert(x) & NotLikes(y, x) ==> Dislikes(y, x)'


]


In [22]:
first_order_KB = FolKB()
for rule in rules:
  first_order_KB.tell(expr(rule))

In [29]:
for r in fol_fc_ask(first_order_KB, expr('Dreams(x, Paris)')):
    print(r)

for r in fol_fc_ask(first_order_KB, expr('Likes(x, NapoleonCake)')):
    print(r)

{x: AuntPolly}
{x: Daddy}
